In [ ]:
# Iterate over the train_generator and attempt to load each image
for i, batch in enumerate(train_generator):
    # Get the images from the batch
    images = batch[0]
    
    # Attempt to load each image
    for j, img_array in enumerate(images):
        try:
            # Convert the image array to a PIL image
            img = Image.fromarray(np.uint8(img_array*255))  # Assuming images are normalized
            
            # Check if the image can be loaded
            img.load()
        except Exception as e:
            # Print the error message along with the filename or index of the problematic image
            print(f"Error loading image at index {i*train_generator.batch_size + j}: {e}")

In [ ]:
train_dataset_folder = f"{dataset_dir}/resized_dataset_train"
test_dataset_folder = f"{dataset_dir}/resized_dataset_test"
train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    preprocessing_function=preprocess_image
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_image
)


train_image_files = [os.path.join(train_dataset_folder, f) for f in os.listdir(train_dataset_folder) if f.endswith(('jpg', 'jpeg', 'png'))]
test_image_files = [os.path.join(test_dataset_folder, f) for f in os.listdir(test_dataset_folder) if f.endswith(('jpg', 'jpeg', 'png'))]

# Create training and validation image generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': train_image_files}),
    x_col='filename',
    class_mode=None,
    target_size=(224, 224),
    batch_size=32,
    shuffle=True
)

validation_generator = test_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': test_image_files}),
    x_col='filename',
    class_mode=None,
    target_size=(224, 224),
    batch_size=32,
    shuffle=False
)

In [ ]:
# Initialize the CNN
model = Sequential()

# Convolutional layer 1
model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional layer 2
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional layer 3
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening
model.add(Flatten())

# Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Output layer

model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.summary()


In [ ]:
# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,  # Adjust epochs based on training time and validation performance
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[early_stopping]
)

loss, accuracy = model.evaluate(test_set)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")